In [ ]:
import sys; sys.path.append('..')
import numpy as np, elastic_rods, linkage_vis
from bending_validation import suppress_stdout as so

l = elastic_rods.RodLinkage('../../examples/florin/20181227_193550_meshID_5ca2f7ab-3602-4ede-ac4d-c2bd798b2961.obj', 8)
driver=l.centralJoint()
l.setMaterial(elastic_rods.RodMaterial('rectangle', 25000, 0.3, [10, 7]))

with so(): elastic_rods.restlen_solve(l)
jdo = l.dofOffsetForJoint(driver)
fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
with so(): elastic_rods.compute_equilibrium(l, fixedVars=fixedVars)
view = linkage_vis.LinkageViewer(l, width=1024, height=640)
view.setCameraParams(((2.3969, -1.1451, -0.5262), (0.0765, 0.0848, -0.9935), (0.1111, 0.1865, 0.5316)))
view.show()

In [ ]:
from open_linkage import open_linkage
def equilibriumSolver(tgtAngle, l, opts, fv):
    opts.gradTol = 1e-4
    return elastic_rods.compute_equilibrium(l, tgtAngle, options=opts, fixedVars=fv)
with so(): open_linkage(l, driver, np.deg2rad(93) - l.averageJointAngle, 40, view, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, useTargetAngleConstraint=True);

# Gravity
Add in downward gravity force (can be done before or after opening the linkage).

In [ ]:
# Use a common density for GFRP
# 2000 kg/m^3 = 2e-6 kg/mm^3
externalForces = l.gradient() + l.gravityForce(2e-6)

fv = l.dofOffsetForJoint(driver) + np.arange(6) # Fix position/orientation at a single joint
with so(): elastic_rods.compute_equilibrium(l, externalForces, fixedVars=fv)

view.update()